# Graph Model of the Million Song Dataset

*Andrea Soto*  
*MIDS W205 Final Project*

# Download the Subset Data - 10,000 songs

In [ ]:
#Create a project directory
!mkdir msd_project
!cd msd_project

In [2]:
%%writefile download_subsetdata.sh
#!/usr/bin/env bash

#Create a directory for the data
mkdir data
cd data

# Download data subset of 10,000 songs, ~1GB to develop and test code
wget http://static.echonest.com/millionsongsubset_full.tar.gz data_subset
wait

tar xvzf millionsongsubset_full.tar.gz
wait

# Download list of all artist ID 
# The format is: artist id<SEP>artist mbid<SEP>track id<SEP>artist name
wget http://labrosa.ee.columbia.edu/millionsong/sites/default/files/AdditionalFiles/unique_artists.txt
wait
wc -l unique_artists.txt #44745 unique_artists.txt

# Download list of all unique artist terms (Echo Nest tags) 
wget http://labrosa.ee.columbia.edu/millionsong/sites/default/files/AdditionalFiles/unique_terms.txt
wait
wc -l unique_terms.txt #7643 unique_terms.txt
    
# Download list of all unique artist musicbrainz tags
wget http://labrosa.ee.columbia.edu/millionsong/sites/default/files/AdditionalFiles/unique_mbtags.txt
wait
wc -l unique_mbtags.txt #2321 unique_mbtags.txt

cd ..

Writing download_subsetdata.sh


# Data Preparation

The Million Song Dataset is stored in HDF5 files. The data was transformed into csv files which can then be used in Neo4j to create the nodes and relationships of the graph.

In [ ]:
# List the files and store them in a separate .txt file
import glob

song_files = glob.glob('./data/MillionSongSubset/data/*/*/*/*.h5')
list_file = './data/list_files.txt'

with open(list_file,'w') as f:
    f.writelines('\n'.join(p for p in song_files))
    f.close()

Start Spark
---

In [ ]:
import os
import sys
#Escape L for line numbers
spark_home = os.environ['SPARK_HOME'] = '/data/spark15'
if not spark_home:
    raise ValueError('SPARK_HOME enviroment variable is not set')

sys.path.insert(0,os.path.join(spark_home,'python'))
sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.8.2.1-src.zip'))
execfile(os.path.join(spark_home,'python/pyspark/shell.py'))

In [ ]:
sc

In [ ]:
file_paths = sc.textFile('file:///data/asoto/projectW205/data/list_files.txt')
#file_paths = sc.textFile('/data/asoto/projectW205/data/list_files.txt')

In [ ]:
file_paths.take(3)

In [ ]:
import h5py

def read_h5_file(path):
    with h5py.File(path, 'r') as f:
        return f['metadata']['songs']['title'][0]

In [ ]:
songs = file_paths.map(read_h5_file)
songs.count()

In [1]:
import time

In [2]:
t1 = time.time()

In [3]:
t2 = time.time()

In [12]:
%%writefile test_code/count_h5.py
#!/usr/bin/env python
from pyspark import SparkContext
import time
import h5py

def read_h5_file(path):
    with h5py.File(path, 'r') as f:
        return f['metadata']['songs']['title'][0]
#Start Time
t1 = time.time()

# --- Process files ----
sc = SparkContext(appName="SparkHDF5")
file_paths = sc.textFile('file:///data/asoto/projectW205/data/list_files.txt')

songs = file_paths.map(read_h5_file)
songs.count()
# ----------------------

#End Time
t2 = time.time()
sec = t2-t1

print "Time: %0.2f sec = %.2f min = %.2f h"%(sec,sec/60.0,sec/1440.0)
sc.stop()

Overwriting test_code/count_h5.py


In [ ]:
!spark-submit test_code/count_h5.py

15/11/29 07:08:53 INFO spark.SparkContext: Running Spark version 1.3.0
15/11/29 07:08:53 INFO spark.SecurityManager: Changing view acls to: asoto
15/11/29 07:08:53 INFO spark.SecurityManager: Changing modify acls to: asoto
15/11/29 07:08:53 INFO spark.SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users with view permissions: Set(asoto); users with modify permissions: Set(asoto)
15/11/29 07:08:54 INFO slf4j.Slf4jLogger: Slf4jLogger started
15/11/29 07:08:54 INFO Remoting: Starting remoting
15/11/29 07:08:54 INFO Remoting: Remoting started; listening on addresses :[akka.tcp://sparkDriver@ip-10-149-10-206.ec2.internal:60992]
15/11/29 07:08:54 INFO Remoting: Remoting now listens on addresses: [akka.tcp://sparkDriver@ip-10-149-10-206.ec2.internal:60992]
15/11/29 07:08:54 INFO util.Utils: Successfully started service 'sparkDriver' on port 60992.
15/11/29 07:08:54 INFO spark.SparkEnv: Registering MapOutputTracker
15/11/29 07:08:54 INFO spark.SparkEnv: Registerin